# Hands-on example

In [1]:
import os
import warnings
import sys

import pandas as pd
import numpy as np

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet

import mlflow
import mlflow.sklearn

## 0. The data

* The data set used in this example is from http://archive.ics.uci.edu/ml/datasets/Wine+Quality
* P. Cortez, A. Cerdeira, F. Almeida, T. Matos and J. Reis.
* Modeling wine preferences by data mining from physicochemical properties. In Decision Support Systems, Elsevier, 47(4):547-553, 2009.

In [2]:
data_path = "data/wine-quality.csv"
data = pd.read_csv(data_path)

data.sample(10)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
2180,8.0,0.360,0.43,10.1,0.053,29.0,146.0,0.99820,3.40,0.46,9.5,6
1593,8.6,0.160,0.49,7.3,0.043,9.0,63.0,0.99530,3.13,0.59,10.5,6
76,7.1,0.180,0.36,1.4,0.043,31.0,87.0,0.98980,3.26,0.37,12.7,7
4826,6.0,0.340,0.29,6.1,0.046,29.0,134.0,0.99462,3.48,0.57,10.7,6
2773,7.6,0.390,0.32,3.6,0.035,22.0,93.0,0.99144,3.08,0.60,12.5,7
1395,6.9,0.190,0.33,1.6,0.039,27.0,98.0,0.98980,3.09,0.46,12.3,7
1035,8.8,0.170,0.38,1.8,0.040,39.0,148.0,0.99420,3.16,0.67,10.2,6
2306,5.6,0.295,0.20,2.2,0.049,18.0,134.0,0.99378,3.21,0.68,10.0,5
1218,7.2,0.260,0.44,7.1,0.027,25.0,126.0,0.99300,3.02,0.34,11.1,8
2458,7.8,0.320,0.33,10.4,0.031,47.0,194.0,0.99692,3.07,0.58,9.6,6


## 1. Tracking experiments

Start the MLflow tracking server by 

```
mlflow server \
    --backend-store-uri /mnt/persistent-disk \
    --default-artifact-root s3://my-mlflow-bucket/ \
    --host 0.0.0.0
    --port 5000
```

or use the default storage method to write to `mlruns/`.

In [5]:
# mlflow server --backend-store-uri mlruns/ --default-artifact-root mlruns/ --host 0.0.0.0 --port 5000
remote_server_uri = "http://127.0.0.1:5000" # set to your server URI
mlflow.set_tracking_uri(remote_server_uri)  # or set the MLFLOW_TRACKING_URI in the env

In [6]:
mlflow.tracking.get_tracking_uri()

'http://127.0.0.1:5000'

In [7]:
exp_name = "ElasticNet_wine"
mlflow.set_experiment(exp_name)

2023/10/16 15:56:48 INFO mlflow.tracking.fluent: Experiment with name 'ElasticNet_wine' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/174255357684808322', creation_time=1697452008226, experiment_id='174255357684808322', last_update_time=1697452008226, lifecycle_stage='active', name='ElasticNet_wine', tags={}>

In [8]:
def eval_metrics(actual, pred):
    # compute relevant metrics
    rmse = np.sqrt(mean_squared_error(actual, pred))
    mae = mean_absolute_error(actual, pred)
    r2 = r2_score(actual, pred)
    return rmse, mae, r2


def load_data(data_path):
    data = pd.read_csv(data_path)

    # Split the data into training and test sets. (0.75, 0.25) split.
    train, test = train_test_split(data)

    # The predicted column is "quality" which is a scalar from [3, 9]
    train_x = train.drop(["quality"], axis=1)
    test_x = test.drop(["quality"], axis=1)
    train_y = train[["quality"]]
    test_y = test[["quality"]]
    return train_x, train_y, test_x, test_y


def train(alpha=0.5, l1_ratio=0.5):
    # train a model with given parameters
    warnings.filterwarnings("ignore")
    np.random.seed(40)

    # Read the wine-quality csv file (make sure you're running this from the root of MLflow!)
    data_path = "data/wine-quality.csv"
    train_x, train_y, test_x, test_y = load_data(data_path)

    # Useful for multiple runs (only doing one run in this sample notebook)    
    with mlflow.start_run():
        # Execute ElasticNet
        lr = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=42)
        lr.fit(train_x, train_y)

        # Evaluate Metrics
        predicted_qualities = lr.predict(test_x)
        (rmse, mae, r2) = eval_metrics(test_y, predicted_qualities)

        # Print out metrics
        print("Elasticnet model (alpha=%f, l1_ratio=%f):" % (alpha, l1_ratio))
        print("  RMSE: %s" % rmse)
        print("  MAE: %s" % mae)
        print("  R2: %s" % r2)

        # Log parameter, metrics, and model to MLflow
        mlflow.log_param(key="alpha", value=alpha)
        mlflow.log_param(key="l1_ratio", value=l1_ratio)
        mlflow.log_metric(key="rmse", value=rmse)
        mlflow.log_metrics({"mae": mae, "r2": r2})
        mlflow.log_artifact(data_path)
        print("Save to: {}".format(mlflow.get_artifact_uri()))
        
        mlflow.sklearn.log_model(lr, "model")

In [9]:
train(0.5, 0.5)

Elasticnet model (alpha=0.500000, l1_ratio=0.500000):
  RMSE: 0.8222428497595401
  MAE: 0.6278761410160693
  R2: 0.12678721972772666
Save to: mlflow-artifacts:/174255357684808322/961cd55c34ad423db736ae9277b2a83e/artifacts


In [10]:
train(0.2, 0.2)

Elasticnet model (alpha=0.200000, l1_ratio=0.200000):
  RMSE: 0.7859129997062342
  MAE: 0.6155290394093895
  R2: 0.20224631822892092
Save to: mlflow-artifacts:/174255357684808322/14e9fd7f50ec40d6a09acde614d4149a/artifacts


In [11]:
train(0.1, 0.1)

Elasticnet model (alpha=0.100000, l1_ratio=0.100000):
  RMSE: 0.7792546522251949
  MAE: 0.6112547988118587
  R2: 0.2157063843066196
Save to: mlflow-artifacts:/174255357684808322/fb66938ecfd24419826c7acfe2d80955/artifacts


### 1.1 Comparing runs
Run `mlflow ui` in a terminal or `http://http://127.0.0.1:5000` to view the experiment log and visualize and compare different runs and experiments. The logs and the model artifacts are saved in the `mlruns` directory (or where you specified).

## 2. Packaging the experiment as a MLflow project as conda env

Specify the entrypoint for this project by creating a `MLproject` file and adding an conda environment with a `conda.yaml`. You can copy the yaml file from the experiment logs.

To run this project, invoke `mlflow run . -P alpha=0.42`. After running this command, MLflow runs your training code in a new Conda environment with the dependencies specified in `conda.yaml`.


## 3. Tagging runs

In [13]:
from datetime import datetime
from mlflow.tracking import MlflowClient

client = MlflowClient()
experiments = client.search_experiments() # returns a list of mlflow.entities.Experiment
print(experiments)

[<Experiment: artifact_location='mlflow-artifacts:/174255357684808322', creation_time=1697452008226, experiment_id='174255357684808322', last_update_time=1697452008226, lifecycle_stage='active', name='ElasticNet_wine', tags={}>, <Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1697451784669, experiment_id='0', last_update_time=1697451784669, lifecycle_stage='active', name='Default', tags={}>]


In [14]:
# get the run
_run = client.get_run(run_id="fb66938ecfd24419826c7acfe2d80955")
print(_run)

<Run: data=<RunData: metrics={'mae': 0.6112547988118587,
 'r2': 0.2157063843066196,
 'rmse': 0.7792546522251949}, params={'alpha': '0.1', 'l1_ratio': '0.1'}, tags={'mlflow.log-model.history': '[{"run_id": "fb66938ecfd24419826c7acfe2d80955", '
                             '"artifact_path": "model", "utc_time_created": '
                             '"2023-10-16 10:29:04.530529", "flavors": '
                             '{"python_function": {"model_path": "model.pkl", '
                             '"predict_fn": "predict", "loader_module": '
                             '"mlflow.sklearn", "python_version": "3.11.4", '
                             '"env": {"conda": "conda.yaml", "virtualenv": '
                             '"python_env.yaml"}}, "sklearn": '
                             '{"pickled_model": "model.pkl", '
                             '"sklearn_version": "1.2.2", '
                             '"serialization_format": "cloudpickle", "code": '
                             'n

In [15]:
# add a tag to the run
dt = datetime.now().strftime("%d-%m-%Y (%H:%M:%S.%f)")
client.set_tag(_run.info.run_id, "deployed", dt)